In [1]:
import os

In [2]:
%pwd

'F:\\Question-Answering-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'F:\\Question-Answering-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_checkpoint: Path
    evaluation_strategy: str
    save_strategy: str
    num_train_epochs: int
    num_update_step_per_epoch: int
    batch_size: int 
    num_warmup_steps: int
    logging_steps: int

In [6]:
from Question_Answering.constants import *
from Question_Answering.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_checkpoint = config. model_checkpoint,
            evaluation_strategy = config.evaluation_strategy,
            save_strategy = config.save_strategy,
            num_train_epochs = config.num_train_epochs,
            num_update_step_per_epoch = config.num_update_step_per_epoch,
            batch_size = config.batch_size,
            num_warmup_steps = config.num_warmup_steps,
            logging_steps = config.logging_steps
        )
        return model_trainer_config

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import default_data_collator
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from tqdm.auto import tqdm

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

     def train(self):
        train_dataset = datasets.load_from_disk(self.config.train_data_path)
        train_dataset.set_format("torch")

        train_dataloader = DataLoader(
            train_dataset,
            shuffle=True,
            collate_fn=default_data_collator,
            batch_size=self.config.batch_size,
            )


        model = AutoModelForQuestionAnswering.from_pretrained(self.config.model_checkpoint)
        optimizer = AdamW(model.parameters(), lr =2e-5)
        accelerator = Accelerator(fp16=True)
        model, optimizer, train_dataloader = accelerator.prepare(
            model, optimizer, train_dataloader)
         
        num_train_epochs = self.config.num_train_epochs
        num_update_steps_per_epoch = len(train_dataloader)
        num_training_steps = num_train_epochs * num_update_steps_per_epoch

        lr_scheduler = get_scheduler(
            "linear",
            optimizer=optimizer,
            num_warmup_steps=self.config.num_warmup_steps,
            num_training_steps=num_training_steps,
        )

        for epoch in range(num_train_epochs):
            # Training
            model.train()
            for step, batch in enumerate(train_dataloader):
                outputs = model(**batch)
                loss = outputs.loss
                accelerator.backward(loss)

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(self.config.root_dir,"Question-Answering-model", save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(self.config.root_dir,"tokenizer")

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e